In [ ]:
import sys
import random
import numpy as np


def read_file(pdb_path):
    lines_atom = []
    with open(pdb_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            if (line[0] == 'ATOM' and line[2] == 'CA'):
                lines_atom.append(line)

    lines_position = [line[6:9] for line in lines_atom]
    lines_position = np.array(lines_position).astype(float)

    # lines_series = [line[1] for line in line_atom]
    # lines_series = np.array(lines_series).astype(int)

    lines_index = [line[5] for line in lines_atom]
    lines_index = np.array(lines_index).astype(int)

    return lines_index, lines_position



def cal_distance(lines_index, lines_position, threshold=10):
    pair = []
    # all_distance = np.zeros([len(lines_position), len(lines_position)])
    for i in range(len(lines_position)):
        for j in range(len(lines_position)):

            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            # all_distance[i][j] = _distance

            if (_distance<=threshold):
                pair.append([lines_index[i], lines_index[j]])

    return np.array(pair)

def cal_domak(lines_index, pair):

    max_split_value = 0
    _counter = 0

    for current in lines_index:
        int_A = np.sum((pair[:,0]<current) & (pair[:,1]<current))
        int_B = np.sum((pair[:,0]>current) & (pair[:,1]>current))
        ext_AB = np.sum((pair[:,0]<=current) & (pair[:,1]>=current))

        if ext_AB != 0:
            _score = (int_A/ext_AB) * (int_B/ext_AB)

            if max_split_value < _score:
                max_split_value = _score
                split_index = current
                index = _counter
        _counter+=1
    
    return max_split_value, split_index, index

# pdb_path = sys.argv[1]
pdb_path = './2csn.pdb'
lines_index, lines_position = read_file(pdb_path)
pair = cal_distance(lines_index, lines_position)
max_split_value, split_index, index = cal_domak(lines_index, pair)

In [ ]:
import sys
import numpy as np


def read_file(pdb_path):
    lines_atom = []
    with open(pdb_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            if (line[0] == 'ATOM' and line[2] == 'CA'):
                lines_atom.append(line)

    lines_position = [line[6:9] for line in lines_atom]
    lines_position = np.array(lines_position).astype(float)

    lines_index = [line[5] for line in lines_atom]
    lines_index = np.array(lines_index).astype(int)

    return lines_index, lines_position


def cal_distance(lines_index, lines_position, threshold=10):
    pair = []
    # all_distance = np.zeros([len(lines_position), len(lines_position)])
    for i in range(len(lines_position)):
        for j in range(len(lines_position)):

            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            # all_distance[i][j] = _distance

            if (_distance<=threshold):
                pair.append([lines_index[i], lines_index[j]])

    return np.array(pair)



def split_group(lines_index, lines_position):

    def get_swap(group_1, group_2):
        max_count = 0
        for i in group_1.keys():
            _simple_count = 0
            for j in group_2.keys():
                _distance = np.sqrt(np.sum( (group_1[i]-group_2[j])**2 ))
                if (_distance<=threshold):
                    _simple_count += 1

            if max_count < _simple_count:
                max_count = _simple_count
                _swap = i
        return max_count, _swap


    dic_position = dict(zip(lines_index, lines_position))
    _split = int(len(lines_position)/2)
    threshold = 10

    sorted = []

    group_U = dict(list(dic_position.items())[:_split] )
    group_V = dict(list(dic_position.items())[_split:] )

    for _ in range(int(len(lines_position)/2)):
        print('\r' + 'Processing {:.2f}%'.format(_/len(lines_position)*100), end='', flush=True)
        max_count1, _swap1 = get_swap(group_U,group_V)
        max_count2, _swap2 = get_swap(group_V,group_U)
        
        if max_count1 > max_count2:
            group_U.pop(_swap1)
            group_V[_swap1] = dic_position[_swap1]

            
        else:
            group_V.pop(_swap2)
            group_U[_swap2] = dic_position[_swap2]

        sorted.append(_swap1)
        sorted.append(_swap2)

    
    best_combine1 = list(set(list(group_U.keys())))
    best_combine2 = list(set(list(group_V.keys())))
            
    print()
    print("Group 1")
    print( best_combine1 )
    print("Group 2")
    print( best_combine2 )

    return best_combine1, best_combine2

pdb_path = './2csn.pdb'
lines_index, lines_position = read_file(pdb_path)
pair = cal_distance(lines_index, lines_position)
best_combine1, best_combine2 = split_group(lines_index, lines_position)

In [23]:
import sys
import numpy as np
import matplotlib.pyplot as plt

def read_file(pdb_path):
    lines_atom = []
    with open(pdb_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            if (line[0] == 'ATOM' and line[2] == 'CA'):
                lines_atom.append(line)

    lines_position = [line[6:9] for line in lines_atom]
    lines_position = np.array(lines_position).astype(float)

    lines_index = [line[5] for line in lines_atom]
    lines_index = np.array(lines_index).astype(int)

    return lines_index, lines_position

def cal_distance(lines_index, lines_position, threshold=10):
    pair = []
    all_distance = np.zeros([len(lines_position), len(lines_position)])
    for i in range(len(lines_position)):
        for j in range(len(lines_position)):
            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            all_distance[i][j] = _distance
            if (i!=j and _distance<=threshold):
                pair.append([lines_index[i], lines_index[j]])
    return np.array(pair)



def cal_domak(lines_index, pair):

    max_split_value = 0
    _counter = 0

    for current in lines_index:
        int_A = np.sum((pair[:,0]<current) & (pair[:,1]<current))
        int_B = np.sum((pair[:,0]>current) & (pair[:,1]>current))
        ext_AB = np.sum((pair[:,0]<=current) & (pair[:,1]>=current))

        if ext_AB != 0:
            _score = (int_A/ext_AB) * (int_B/ext_AB)

            if max_split_value < _score:
                max_split_value = _score
                split_index = current
                index = _counter
        _counter+=1
    
    return max_split_value, split_index, index


def get_all_split(lines_index, lines_position, indexs):
    min_residue = 1
    min_split_score = 1
    
    pair = cal_distance(lines_index, lines_position)
    max_split_value, split_index, index = cal_domak(lines_index, pair)
    


    lines_index1 = lines_index[split_index:]
    lines_index2 = lines_index[:split_index]

    lines_position1 = lines_position[split_index:]
    lines_position2 = lines_position[:split_index]

    if max_split_value < min_split_score: return 
    if len(lines_position1) < min_residue: return
    if len(lines_position2) < min_residue: return

    indexs.append(index)

    get_all_split(lines_index2, lines_position2, indexs)
    get_all_split(lines_index1, lines_position1, indexs)

def make_fingure(pair, splits):
    plt.figure(figsize=(8, 8))
    plt.plot(pair[:, 0], pair[:, 1], 'o', markersize=1)

    colors = 'rgbycmkw'
    for inx, split in enumerate(splits):
        plt.axvline(split, color = colors[inx])
        plt.axhline(split, color = colors[inx])
    plt.show()


pdb_path = './2csn.pdb'
lines_index, lines_position = read_file(pdb_path)
indexs = []
get_all_split(lines_index, lines_position, indexs)
indexs

[82, 8]

In [28]:
import sys
import numpy as np
import matplotlib.pyplot as plt

def read_file(pdb_path):
    lines_atom = []
    with open(pdb_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            if (line[0] == 'ATOM' and line[2] == 'CA'):
                lines_atom.append(line)

    lines_position = [line[6:9] for line in lines_atom]
    lines_position = np.array(lines_position).astype(float)

    lines_index = [line[5] for line in lines_atom]
    lines_index = np.array(lines_index).astype(int)

    return lines_index, lines_position

def cal_distance(lines_index, lines_position, threshold=10):
    pair = []
    all_distance = np.zeros([len(lines_position), len(lines_position)])
    for i in range(len(lines_position)):
        for j in range(len(lines_position)):
            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            all_distance[i][j] = _distance
            if (i!=j and _distance<=threshold):
                pair.append([lines_index[i], lines_index[j]])
    return np.array(pair)



def cal_domak(lines_index, pair):

    max_split_value = 0
    _counter = 0

    for current in lines_index:
        int_A = np.sum((pair[:,0]<current) & (pair[:,1]<current))
        int_B = np.sum((pair[:,0]>current) & (pair[:,1]>current))
        ext_AB = np.sum((pair[:,0]<=current) & (pair[:,1]>=current))

        if ext_AB != 0:
            _score = (int_A/ext_AB) * (int_B/ext_AB)

            if max_split_value < _score:
                max_split_value = _score
                split_index = current
                index = _counter
        _counter+=1
    
    return max_split_value, split_index, index


def get_all_split(lines_index, lines_position, indexs):
    min_residue = 1
    min_split_score = 1
    
    pair = cal_distance(lines_index, lines_position)
    max_split_value, split_index, index = cal_domak(lines_index, pair)

    lines_index1 = lines_index[split_index:]
    lines_index2 = lines_index[:split_index]

    lines_position1 = lines_position[split_index:]
    lines_position2 = lines_position[:split_index]

    if max_split_value < min_split_score: return 
    if len(lines_position1) < min_residue: return
    if len(lines_position2) < min_residue: return

    indexs.append(split_index)

    get_all_split(lines_index1, lines_position1, indexs)
    get_all_split(lines_index2, lines_position2, indexs)
    

def make_fingure(pair, splits):
    plt.figure(figsize=(8, 8))
    plt.plot(pair[:, 0], pair[:, 1], 'o', markersize=1)

    colors = 'rgbycmkw'
    for inx, split in enumerate(splits):
        plt.axvline(split, color = colors[inx])
        plt.axhline(split, color = colors[inx])
    plt.show()


pdb_path = './2csn.pdb'
lines_index, lines_position = read_file(pdb_path)
indexs = []
get_all_split(lines_index, lines_position, indexs)